<a href="https://colab.research.google.com/github/NastuaBudko/machine_learning/blob/main/%D0%91%D1%83%D0%B4%D0%BA%D0%BE_%D0%BF%D1%80_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Будко А.С. ФІТ 4-8 Варіант 4

# Завдання 1

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357283 sha256=82aa6dbe2d3d5fe353074169eac3ebfc55d694c8b66036a4dcd4bd7038f4b57d
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
from surprise import Dataset, Reader
from surprise import SVD, KNNBasic
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import accuracy
from surprise.model_selection import train_test_split

1. Завантажте датасет для рецензій (ml-100k) за допомогою бібліотеки
Surprise.

In [3]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


2. Виведіть перші 5 рядків завантаженого датасету.

In [4]:
for row in data.raw_ratings[:5]:
    print(row)

('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


3. Реалізуйте два алгоритми для рекомендаційної системи на основі цього
датасету. Можна вибрати будь-які алгоритми з бібліотеки Surprise.

In [5]:
trainset, testset = train_test_split(data, test_size=0.2)

param_grid_svd = {'n_factors': [50, 100],
                  'lr_all': [0.005, 0.010],
                  'reg_all': [0.02, 0.05]}

param_grid_knn = {'k': [20, 40],
                  'min_k': [1, 5],
                  'sim_options': {'name': ['cosine', 'pearson'],
                                  'user_based': [False, True]}}

4. Використайте крос-валідацію для підбору оптимальних параметрів для
обох алгоритмів.

In [6]:
grid_search_svd = GridSearchCV(SVD, param_grid_svd, measures=['rmse', 'mae'], cv=5)
grid_search_svd.fit(data)

grid_search_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['rmse', 'mae'], cv=5)
grid_search_knn.fit(data)


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing

5. Оберіть найкращий алгоритм на основі середньої абсолютної помилки
(MAE).


In [7]:
best_svd = grid_search_svd.best_estimator['mae']
best_knn = grid_search_knn.best_estimator['mae']

print(f"Найкращий MAE для SVD: {grid_search_svd.best_score['mae']}")
print(f"Найкращий MAE для KNNBasic: {grid_search_knn.best_score['mae']}")

if grid_search_svd.best_score['mae'] < grid_search_knn.best_score['mae']:
    best_model = best_svd
    print("Вибраний алгоритм: SVD")
else:
    best_model = best_knn
    print("Вибраний алгоритм: KNNBasic")

Найкращий MAE для SVD: 0.7247255159781327
Найкращий MAE для KNNBasic: 0.8033514364551919
Вибраний алгоритм: SVD


6. Виведи рекомендації (10 фільмів) для конкретного користувача.


In [8]:
best_model.fit(trainset)

user_id = str(196)
user_ratings = trainset.ur[trainset.to_inner_uid(user_id)]
print(f"Кількість рецензій користувача {user_id}: {len(user_ratings)}")

all_items = set(trainset.all_items())
rated_items = set([item for (item, _) in user_ratings])
unrated_items = all_items - rated_items

predictions = [(item, best_model.predict(user_id, trainset.to_raw_iid(item)).est) for item in unrated_items]
predictions.sort(key=lambda x: x[1], reverse=True)

print("Топ-10 фільмів, рекомендованих для користувача:")
for item_id, rating in predictions[:10]:
    print(f"Фільм {trainset.to_raw_iid(item_id)} з прогнозованим рейтингом {rating:.2f}")

Кількість рецензій користувача 196: 36
Топ-10 фільмів, рекомендованих для користувача:
Фільм 408 з прогнозованим рейтингом 4.61
Фільм 480 з прогнозованим рейтингом 4.51
Фільм 512 з прогнозованим рейтингом 4.42
Фільм 479 з прогнозованим рейтингом 4.42
Фільм 1449 з прогнозованим рейтингом 4.39
Фільм 481 з прогнозованим рейтингом 4.36
Фільм 1021 з прогнозованим рейтингом 4.33
Фільм 166 з прогнозованим рейтингом 4.31
Фільм 320 з прогнозованим рейтингом 4.30
Фільм 493 з прогнозованим рейтингом 4.30


Висновок: У рамках цієї лабораторної роботи я реалізувала два алгоритми для рекомендаційної системи на основі датасету "ml-100k", провів крос-валідацію для кожного з них та відобразила рекомендації 10 фільмів для вибраного користувача.